In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

#Shrinking Dataset Down (We can change these numbers)

In [2]:
#Our cutoffs for plate appearances in a matchup, then total matchup cutoffs
#4 and 4 was causing memory problems
#12 and 12 did not
#PA_cutoff = 12
#Match_cutoff = 12

#Shrinking the data size a bit
#data = pd.read_csv("temp_matchupdf.csv")
#filtering out matchups that have less than 4 plate appearances
#data = data[data.PA >= PA_cutoff]
#count = data.groupby('bID').size()

In [3]:
#countdf=count.to_frame(name='bID_count').reset_index()
#data = data.merge(countdf, how='left')
#filtering out batters that have less than 4 matchups
#data = data[data.bID_count >= Match_cutoff]

In [4]:
#data = data[['bID','pID','AVG', 'OBP']]

In [5]:
#data.shape

In [6]:
#data = data.dropna()

In [7]:
#data.shape

#Test-Train Split

In [2]:
smalldf = pd.read_csv("small.csv")

In [57]:
#temporary test-train split - group by batter id
#Need to determine how many of each to put in the test and training set. Going to take 2 for validation, 1 for test for now

trainlist=[]
testlist=[]
validatelist=[]
take=5
for k, v in smalldf.groupby('bID'):
    if np.mean(v.batter_PA_count) > 150:
        takenos=np.random.choice(range(take), size=take/2, replace=False)
        takelist=np.array([e in takenos for e in range(take)])
        validatelist.append(v[-take:][~takelist])#use those 
        testlist.append(v[-take:][takelist])#use the other 
        trainlist.append(v[:-take])
    else:
        trainlist.append(v)
traindf=pd.concat(trainlist)
validatedf=pd.concat(validatelist)
testdf=pd.concat(testlist)
print traindf.shape, validatedf.shape, testdf.shape

(235033, 34) (1743, 34) (1162, 34)


#Make the uuids and uiids from the traindf for each batter and pitcher

In [4]:
ubids=traindf.bID.unique()#unique-user-ids
upids=traindf.pID.unique()#unique-item-ids

In [5]:
from sklearn.linear_model import Ridge
features=np.concatenate([ubids,upids])
features.shape

(2887L,)

In [6]:
traindf.head()

,bID,pID,AB,H,PA,RL,SAC,SO,TB,W,AVG,OBP,ov_AVG,ov_OBP,ov_PA,ov_AB,ov_H,ov_W,ov_TB,ov_SAC,ovp_SO_PCT,ovp_W_PCT,ovp_H_PCT,ovp_SAC,ovp_SO,ovp_AB,ovp_H,ovp_W,opv_TB,ovp_PA,pitcher_avg,pitcher_PA_count,batter_avg,batter_PA_count
165077,aberb001,armat002,4,1,4,R,0,1,1,0,0.250000,0.250000,0.238325,0.287651,664,621,148,43,231,9,0.158114,0.129570,0.232200,54,493,2714,724,404,1512,3118,0.257677,286,0.240567,85
165078,aberb001,baldj002,6,2,6,R,0,0,6,0,0.333333,0.333333,0.238325,0.287651,664,621,148,43,231,9,0.140293,0.096919,0.255160,38,469,3019,853,324,1733,3343,0.287744,239,0.240567,85
165079,aberb001,bauer001,4,0,4,R,0,0,0,0,0.000000,0.000000,0.238325,0.287651,664,621,148,43,231,9,0.117284,0.113580,0.253086,9,95,718,205,92,372,810,0.289811,118,0.240567,85
165080,aberb001,bellr003,9,1,10,R,0,0,1,1,0.111111,0.200000,0.238325,0.287651,664,621,148,43,231,9,0.128057,0.107603,0.264562,35,288,2007,595,242,1355,2249,0.285844,234,0.240567,85
165081,aberb001,benoj001,4,1,5,R,0,0,1,1,0.250000,0.400000,0.238325,0.287651,664,621,148,43,231,9,0.209011,0.117886,0.213076,41,617,2604,629,348,1293,2952,0.238144,326,0.240567,85


In [52]:
#function to produce the design matrix
def getmats(indf):
    
    #we can adjust the values that we want getmats to spit out - here we do average and on base percentage
    OBP = indf['OBP']
    
    n_cols = len(features)
    n_rows = len(indf)
    
    #fill matrix with zeros
    mats = np.zeros((n_rows, n_cols))

    #for each element in features, check for match in every row, whether business or user, and switch to one
    for i in range(0, len(ubids)):
        for j in range(0, n_rows):
            if features[i] == indf['bID'].values[j]:
                mats[j, i] = 1
    for i in range(len(ubids), n_cols):
        for j in range(0, n_rows):
            if features[i] == indf['pID'].values[j]:
                mats[j, i] = 1
    return mats, OBP

In [53]:
%%time
designm, obp = getmats(traindf)
validatedm, validate_obp = getmats(validatedf)

MemoryError: 

In [10]:
def get_rmse(s, s_predict):
    diff = s - s_predict
    return np.sqrt(np.dot(diff,diff)/diff.shape[0])

In [11]:
%%time

#regressing averages
from sklearn.linear_model import Ridge
alphas = [0.01, 0.1, 10, 100, 1000]
vdict={}
rdict={}
for a in alphas:
    #perform regression for each alpha and store in rdict, keyed by the alpha
    ridge_reg = Ridge(alpha=a)
    ridge_reg = ridge_reg.fit(designm, avg)
    rdict[a]=ridge_reg
    
    #calculate the RMSE for each alpha, store in vdict
    pred = ridge_reg.predict(validatedm)
    rmse = get_rmse(pred, validate_avg)
    vdict[a]=rmse

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [30]:
vdict

{0.01: 0.11294228728430696,
 0.1: 0.11291557169765709,
 10: 0.11220503169451662,
 100: 0.11258549799111771,
 1000: 0.11359314923967566}

In [31]:
minerroralpha=min(vdict, key=vdict.get)
print minerroralpha
regr=rdict[minerroralpha] 

10


In [32]:
regr.intercept_

0.27009153451168733

In [33]:
meancept=regr.intercept_
fitratmap_b={k:regr.coef_[i] for i,k in enumerate(ubids)}
fitratmap_p={k:regr.coef_[i+len(ubids)] for i,k in enumerate(upids)}

In [35]:
#train_fits={'mean':meancept, 'users':fitratmap_u, 'items':fitratmap_i}

#Prediction

In [ ]:
testdm, test_avg = getmats(testdf)

In [ ]:
from sklearn.linear_model import Ridge

pred_test = regr.predict(testdm)
predictions['baseline_r'] = pred_test

pred_valid = regr.predict(validatedm)
predictions_valid['baseline_r'] = pred_valid

compare_results(test_avg,predictions['baseline_r'], model="baseline_r", predicteds=True);

In [ ]:
ax=plt.gca()
compare_results(test_avg,predictions['baseline_r'], model="baseline_r", predicteds=True, axis=ax);
compare_results(testdf.stars,predictions['baseline'], model="baseline", predicteds=True, onsame=ax);